In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

clear_output()

In [2]:
df = pd.read_pickle('data/saeb_treated.pkl')
df.head()

,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,proficiencia,erro_padrao,proficiencia_saeb,erro_padrao_saeb,...,escolaridade_pai_4,escolaridade_pai_5,escolaridade_pai_6,escolaridade_pai_7,possui_computador_0,possui_computador_1,possui_computador_2,possui_computador_3,possui_computador_4,possui_computador_5
274,2015,PB,1,17518688,0,MT,-0.472096,0.538400,223.577837,30.092429,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
348,2015,PB,1,17518163,0,MT,-0.809452,0.580177,204.722224,32.427443,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
408,2015,PB,1,17491526,0,LP,-0.828752,0.366511,204.326255,20.192323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
417,2015,PB,1,17523365,0,LP,-0.564052,0.351783,218.909469,19.380908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1008,2015,PB,1,17508044,0,LP,-0.775107,0.449797,207.281739,24.780829,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
df = df.dropna(subset=['desempenho_aluno_cat'])
print(df.shape)

(133774, 66)


In [4]:
df = df[df['disciplina'] == 'MT']

In [5]:
print(df.columns)

Index(['ano', 'sigla_uf', 'escola_publica', 'id_aluno', 'situacao_censo',
       'disciplina', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
       'erro_padrao_saeb', 'desempenho_aluno', 'sexo', 'raca_cor',
       'ano_nascimento', 'idade', 'mae_sabe_ler_escrever', 'mae_le',
       'pai_sabe_ler_escrever', 'pai_le',
       'responsaveis_incentivam_realizacao_licao_casa',
       'responsaveis_incentivam_estudos', 'responsaveis_incentivam_leitura',
       'responsaveis_incentivam_comparecer_aulas', 'tempo_lazer',
       'leitura_jornais', 'leitura_livros_geral',
       'leitura_literatura_infantojuvenil', 'leitura_historia_quadrinhos',
       'leitura_revista_comportamento', 'leitura_revistas_geral',
       'leitura_internet', 'frequenta_biblioteca', 'frequenta_cinema',
       'frequenta_espetaculo_exposicao', 'frequenta_festas_comunidade',
       'gosta_estudar_disciplina', 'faz_licao_casa', 'correcao_licao_casa',
       'utiliza_biblioteca_escola', 'rede', 'id_escola', 'serie', 

In [6]:
exclude = ['ano', 'sigla_uf', 'id_aluno', 'situacao_censo', 'disciplina', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
       'erro_padrao_saeb', 'desempenho_aluno', 'sexo', 'raca_cor','ano_nascimento', 'rede', 'id_escola', 'serie', 'turno']

In [7]:
# print unique values for each column excluding exclude columns
for col in df.columns:
    if col not in exclude:
        print(col, df[col].unique())

escola_publica <IntegerArray>
[1, 0]
Length: 2, dtype: Int64
idade [15 18 17 16 14 13]
mae_sabe_ler_escrever [1 0 2]
mae_le [1 2 0]
pai_sabe_ler_escrever [1 0 2]
pai_le [1 0 2]
responsaveis_incentivam_realizacao_licao_casa [ 0. nan  1.]
responsaveis_incentivam_estudos [ 0. nan  1.]
responsaveis_incentivam_leitura [ 0.  1. nan]
responsaveis_incentivam_comparecer_aulas [ 0. nan  1.]
tempo_lazer [ 4.  1.  2.  3.  0. nan]
leitura_jornais [ 1.  2.  0. nan]
leitura_livros_geral [ 2.  1.  0. nan]
leitura_literatura_infantojuvenil [ 2.  0.  1. nan]
leitura_historia_quadrinhos [ 1. nan  0.  2.]
leitura_revista_comportamento [ 1.  2. nan  0.]
leitura_revistas_geral [ 2.  0.  1. nan]
leitura_internet [ 0.  2.  1. nan]
frequenta_biblioteca [ 2.  1.  0. nan]
frequenta_cinema [ 2.  1.  0. nan]
frequenta_espetaculo_exposicao [ 2.  1. nan  0.]
frequenta_festas_comunidade [ 1.  0.  2. nan]
gosta_estudar_disciplina [ 0.  1. nan]
faz_licao_casa [0 1 2 4 3]
correcao_licao_casa [0 4 1 3 2]
utiliza_bibliote

Filtering unimportant columns

In [14]:
# colunas:
# idade, leitura_jornais, leitura_revista_comportamento,
# escolaridade_mae_0, escolaridade_mae_1, escolaridade_mae_4, escolaridade_mae_5, escolaridade_mae_6
# escolaridade_pai_0, escolaridade_pai_4, escolaridade_pai_5
# possui_computador_0, possui_computador_1, possui_computador_2
# mae_sabe_ler_escrever, pai_sabe_ler_escrever

# conjunto 1:
# idade, frequenta_espetaculo_exposicao, frequenta_festas_comunidade, frequenta_biblioteca
# tempo_lazer, gosta_estudar_disciplina, faz_licao_casa, correcao_licao_casa, utiliza_biblioteca_escola

# conjunto 2:
# idade, leitura_jornais, leitura_revista_comportamento, mae_sabe_ler_escrever, pai_sabe_ler_escrever
# frequenta_biblioteca, utiliza_biblioteca_escola

# conjunto 3:
# idade, escolaridade_mae_0, escolaridade_mae_1, escolaridade_mae_4, escolaridade_mae_5, escolaridade_mae_6
# escolaridade_pai_0, escolaridade_pai_4, escolaridade_pai_5, mae_sabe_ler_escrever, pai_sabe_ler_escrever

# conjunto 4:
# idade, faz_licao_casa, correcao_licao_casa, utiliza_biblioteca_escola, gosta_estudar_disciplina

# conjunto 5:
# idade, possui_computador_0, possui_computador_1, possui_computador_2, mae_sabe_ler_escrever, pai_sabe_ler_escrever
# gosta_estudar_disciplina, faz_licao_casa, correcao_licao_casa

In [15]:
df.columns

Index(['ano', 'sigla_uf', 'escola_publica', 'id_aluno', 'situacao_censo',
       'disciplina', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
       'erro_padrao_saeb', 'desempenho_aluno', 'sexo', 'raca_cor',
       'ano_nascimento', 'idade', 'mae_sabe_ler_escrever', 'mae_le',
       'pai_sabe_ler_escrever', 'pai_le',
       'responsaveis_incentivam_realizacao_licao_casa',
       'responsaveis_incentivam_estudos', 'responsaveis_incentivam_leitura',
       'responsaveis_incentivam_comparecer_aulas', 'tempo_lazer',
       'leitura_jornais', 'leitura_livros_geral',
       'leitura_literatura_infantojuvenil', 'leitura_historia_quadrinhos',
       'leitura_revista_comportamento', 'leitura_revistas_geral',
       'leitura_internet', 'frequenta_biblioteca', 'frequenta_cinema',
       'frequenta_espetaculo_exposicao', 'frequenta_festas_comunidade',
       'gosta_estudar_disciplina', 'faz_licao_casa', 'correcao_licao_casa',
       'utiliza_biblioteca_escola', 'rede', 'id_escola', 'serie', 

In [16]:
df['rede'].value_counts()

rede
1    37403
0    27728
2     1756
Name: count, dtype: int64

In [17]:
df['turno'].value_counts()

turno
0    35624
1    30567
2      696
Name: count, dtype: int64

In [18]:
df['desempenho_aluno'].value_counts() # estão desbalanceados

desempenho_aluno
Básico          33972
Insuficiente    27368
Proficiente      4962
Avançado          585
Name: count, dtype: int64

In [41]:
_df = df.dropna()

In [47]:
# aplicando smote
from imblearn.combine import SMOTEENN

X = _df[['frequenta_espetaculo_exposicao', 'frequenta_festas_comunidade', 'frequenta_biblioteca',
        'tempo_lazer', 'gosta_estudar_disciplina', 'faz_licao_casa', 'correcao_licao_casa', 'utiliza_biblioteca_escola']]
y = _df['desempenho_aluno_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Aplicar SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

# Verificar a distribuição das classes após o balanceamento
print(y_resampled.value_counts())

desempenho_aluno_cat
3    10241
1     3977
0     2531
2     2385
Name: count, dtype: int64


In [55]:
_df.drop(columns=exclude + ['desempenho_aluno', 'desempenho_aluno_cat', 'escola_publica']).columns

Index(['idade', 'mae_sabe_ler_escrever', 'mae_le', 'pai_sabe_ler_escrever',
       'pai_le', 'responsaveis_incentivam_realizacao_licao_casa',
       'responsaveis_incentivam_estudos', 'responsaveis_incentivam_leitura',
       'responsaveis_incentivam_comparecer_aulas', 'tempo_lazer',
       'leitura_jornais', 'leitura_livros_geral',
       'leitura_literatura_infantojuvenil', 'leitura_historia_quadrinhos',
       'leitura_revista_comportamento', 'leitura_revistas_geral',
       'leitura_internet', 'frequenta_biblioteca', 'frequenta_cinema',
       'frequenta_espetaculo_exposicao', 'frequenta_festas_comunidade',
       'gosta_estudar_disciplina', 'faz_licao_casa', 'correcao_licao_casa',
       'utiliza_biblioteca_escola', 'escolaridade_mae_0', 'escolaridade_mae_1',
       'escolaridade_mae_2', 'escolaridade_mae_3', 'escolaridade_mae_4',
       'escolaridade_mae_5', 'escolaridade_mae_6', 'escolaridade_mae_7',
       'escolaridade_pai_0', 'escolaridade_pai_1', 'escolaridade_pai_2',
      

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
import numpy as np

X = _df.drop(columns=exclude + ['desempenho_aluno', 'desempenho_aluno_cat', 'escola_publica'])
y = _df['desempenho_aluno_cat']

cv = StratifiedKFold(n_splits=5)

for fold, (train_idx, test_idx) in enumerate(cv.split(X, y), 1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    sm = SMOTE(random_state=42)
    X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
    
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_oversampled, y_train_oversampled)
    
    y_pred = model.predict(X_test)
    
    print(f'For fold {fold}:')
    print(f'Accuracy: {model.score(X_test, y_test)}')
    print(f'f-score: {f1_score(y_test, y_pred, average="weighted")}')

For fold 1:
Accuracy: 0.5280446927374302
f-score: 0.5053299665040977
For fold 2:
Accuracy: 0.5388826815642458
f-score: 0.49713152247244496
For fold 3:
Accuracy: 0.5059217877094973
f-score: 0.4312408513955841
For fold 4:
Accuracy: 0.5440831377807577
f-score: 0.4931204046882748
For fold 5:
Accuracy: 0.5500055872164488
f-score: 0.5224565665853377


In [48]:
# treinar o modelo

print(X_resampled.shape, y_resampled.shape)

model = Sequential()
model.add(Dense(8, input_dim=X_resampled.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))

# desempenho_aluno_cat has 4 classes
model.add(Dense(4, activation='softmax'))

model.summary()

# compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

(19134, 8) (19134,)


/home/erlo/miniconda3/envs/aprendizagem/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 4)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 252 (1008.00 B)

 Trainable params: 252 (1008.00 B)

 Non-trainable params: 0 (0.00 B)

In [49]:
history = model.fit(X_resampled, y_resampled, epochs=100, batch_size=48)

Epoch 1/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 649us/step - accuracy: 0.4912 - loss: 1.2993
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step - accuracy: 0.5834 - loss: 1.0471
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.6128 - loss: 0.9907
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 0.6212 - loss: 0.9673
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.6243 - loss: 0.9466
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.6298 - loss: 0.9271
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 0.6351 - loss: 0.9166
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - accuracy: 0.6290 - loss: 0.9128
Epoch 9/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step - accuracy: 0.6311 - loss: 0.9110
Epoch 10/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - accuracy: 0.6361 - loss: 0.9037
Epoch 11/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - accuracy: 0.6329 - loss: 0.8998
Epoch 12/100
399/39

KeyboardInterrupt: 

In [27]:
# conjunto 2
X = df[['leitura_jornais', 'leitura_revista_comportamento', 'mae_sabe_ler_escrever', 'pai_sabe_ler_escrever',
        'frequenta_biblioteca', 'utiliza_biblioteca_escola']]
y = df['desempenho_aluno_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# aplicando smote
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

model = Sequential()
model.add(Dense(4, input_dim=X.shape[1], activation='relu'))
model.add(Dense(4, activation='relu'))

# desempenho_aluno_cat has 4 classes
model.add(Dense(4, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

model.summary()

# compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/home/erlo/miniconda3/envs/aprendizagem/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 4)              │            28 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 4)              │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 4)              │            20 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68 (272.00 B)

 Trainable params: 68 (272.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
history = model.fit(X_train, y_train, epochs=100, batch_size=48, validation_data=(X_val, y_val), callbacks=[es])

Epoch 1/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4165 - loss: 1.2068 - val_accuracy: 0.5101 - val_loss: 0.9605
Epoch 2/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5095 - loss: 0.9499 - val_accuracy: 0.5101 - val_loss: 0.9505
Epoch 3/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5076 - loss: 0.9469 - val_accuracy: 0.5101 - val_loss: 0.9498
Epoch 4/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5073 - loss: 0.9466 - val_accuracy: 0.5101 - val_loss: 0.9500
Epoch 5/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5074 - loss: 0.9449 - val_accuracy: 0.5101 - val_loss: 0.9499
Epoch 6/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5073 - loss: 0.9420 - val_accuracy: 0.5101 - val_loss: 0.9499
Epoch 7/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5077 - loss: 0.9452 - val_accuracy: 0.5101 - val_loss: 0.9499
Epoch 8/100
892/892 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5031 - loss: 0.9463 - val_accu